In [36]:
import api_secrets
from googleapiclient.discovery import build
from IPython.display import JSON
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [57]:
def get_top_1000_channels_spain():
    '''
    WebScraping of us.youtubers.me to get the top 1000 youtubechannels in Spain
    '''
    usernames = []
    url = "https://us.youtubers.me/spain/all/top-1000-most-subscribed-youtube-channels-in-spain"
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find('table', class_='top-charts')
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        if (columns != []):
            cell = columns[1]
            link = cell.find_all('a', href=True)[0]['href']
            usernames.append(link.split('/')[1])
    return usernames

In [33]:
def get_channel_ids(usernames):
    '''
    Get the channel ids from a list of usernames
    '''
    channel_ids = []
    for user in usernames:
        channel_data = youtube.search().list(
            part="snippet",
            maxResults=1,
            q=user,
            type="channel"
        ).execute()
        channel_ids.append(channel_data['items'][0]['id']['channelId'])
    return channel_ids
        

In [58]:
def get_channel_stats(channel_ids):
    '''
    Get the statistic for all the channels in a list of channel ids.
    Code snippet from https://github.com/thu-vu92/youtube-api-analysis/blob/main/Sample_project_youtube_videos_EDA.ipynb
    Returns:
    - A table to visualize the stats
    - A list with the stats in JSON format
    '''
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute() 
    
    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return pd.DataFrame(all_data), all_data

In [66]:
def get_playlist_ids(channel_stats):
    playlist_ids = []
    for channel in channel_stats:
        playlist_ids.append(channel['playlistId'])
    return playlist_ids

In [67]:
def get_video_ids(playlist_id):
    """
    Get list of video IDs of all videos in the given playlist
    Params:
    
    youtube: the build object from googleapiclient.discovery
    playlist_id: playlist ID of the channel
    
    Returns:
    List of video IDs of all videos in the playlist
    
    Code snippet from https://github.com/thu-vu92/youtube-api-analysis/blob/main/Sample_project_youtube_videos_EDA.ipynb
    """
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [76]:
def get_video_stats(video_ids):
    '''
    Get the statistic for all the video in a list of video ids.
    Code snippet from https://github.com/thu-vu92/youtube-api-analysis/blob/main/Sample_project_youtube_videos_EDA.ipynb
    Returns:
    - A table to visualize the stats
    - A list with the stats in JSON format
    '''
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info), all_video_info

In [62]:
youtube = build("youtube", "v3", developerKey=api_secrets.api_key)
usernames = get_top_1000_channels_spain()

In [123]:
channel_ids = get_channel_ids(usernames[0:2])
channel_stats_table, channel_stats = get_channel_stats(channel_ids)
playlist_ids = get_playlist_ids(channel_stats)
video_ids = []
for playlist in playlist_ids:
    video_ids.append(get_video_ids(playlist))

video_stats_table = pd.DataFrame()
video_stats = []
for videos in video_ids:
    tmp_table, tmp_stats = get_video_stats(videos)
    video_stats_table = pd.concat([video_stats_table, tmp_table])
    video_stats.append(tmp_stats)

In [110]:
channel_stats_table

,channelName,subscribers,views,totalVideos,playlistId
0,Mikecrack,44900000,17157126172,1958,UUqJ5zFEED1hWs0KNQCQuYdQ
1,elrubiusOMG,40300000,7429512908,703,UUXazgXDIYyWH-yXLAkcrFxw


In [111]:
video_stats_table

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,Cyph43FTmlc,Mikecrack,AYUDA: EL JUEGO MÁS ATERRADOR EN AÑOS 😱😨 MI PR...,"Para celebrar Halloween, Mikecrack juega un Fi...","[mikecrack, mike crack, mikecrak, mike, five n...",2023-10-31T21:38:53Z,1268829,62871,None,4756,PT2H5M16S,hd,false
1,RhVBRyYwrJ0,Mikecrack,¡DULCE O TRUCO! 🍬🎃 ESPECIAL de HALLOWEEN 🦇 Las...,Mike y sus amigos están listos para vivir la m...,"[mikecrack, mike crack, mikecrak, mike, hallow...",2023-10-28T18:00:07Z,1809451,81020,None,5390,PT11M48S,hd,false
2,MTOsyCwbRkk,Mikecrack,¡CONSTRUYENDO UN NUEVO HOGAR! 😍🏠 Las Perrerías...,Regresan Las Perrerías de Mike Temporada 3 con...,"[mikecrack, mike crack, mikecrak, mike, las pe...",2023-10-21T20:30:08Z,2237902,87493,None,6674,PT8M3S,hd,false
3,ZMt0rl2g4no,Mikecrack,🎤 TOMA TU MARTILLO 🔨😁 CANCIÓN ORIGINAL de MIKE...,Toma tu martillo es la canción original de Mik...,"[mikecrack, mike crack, mikecrak, mike, toma t...",2023-10-20T20:30:09Z,3224430,84039,None,8242,PT4M27S,hd,false
4,dnOv_pdLQo0,Mikecrack,EL DÍA QUE MIS SUEÑOS SE HICIERON REALIDAD... ...,Hoy os traigo un vlog muy especial en el que o...,"[mikecrack, mike crack, mikecrak, mike, la sup...",2023-10-06T02:49:18Z,1169018,47593,None,4947,PT17M38S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,B81VP6ZidHo,elrubiusOMG,Skyrim - Un regalo para mi mujer,No quiere usarlo D:\r\n\r\nCancion: Dubsidia -...,"[elrubius, rubius, skyrim, mujer, comentario, ...",2012-01-04T06:30:31Z,1409175,41871,None,2409,PT1M35S,hd,true
699,ML5n5kvnbJ8,elrubiusOMG,Skyrim - La hija del Señor Oso,"No se porque tanto cabreo, si solo la acaricié!","[elrubius, rubius, skyrim, tes, señor, oso, ca...",2012-01-04T05:46:59Z,1393255,35706,None,892,PT41S,hd,true
700,b-hITqIkxD8,elrubiusOMG,Skyrim - Curiosidades divertidas -Part 1-,"Estas en pelotas, en pelotas, EN PELOTAS!\r\n\...","[elrubius, rubius, skyrim, tes, curiosidades, ...",2012-01-04T05:30:45Z,2262671,56350,None,2209,PT4M13S,hd,true
701,oD0d5xQCAYI,elrubiusOMG,Skyrim - Eligiendo a la mujer perfecta,No sirve para la cocina D:\r\n\r\nResubo!,"[elrubius, rubius, skyrim, mujer, perfecta, el...",2012-01-04T03:26:31Z,1874067,51255,None,2681,PT1M38S,hd,true


In [135]:
video_stats_table['publishedWeekDay'] = pd.to_datetime(video_stats_table['publishedAt'], errors='coerce').dt.day_name()

In [136]:
video_stats_table

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,publishedWeekDay
0,Cyph43FTmlc,Mikecrack,AYUDA: EL JUEGO MÁS ATERRADOR EN AÑOS 😱😨 MI PR...,"Para celebrar Halloween, Mikecrack juega un Fi...","[mikecrack, mike crack, mikecrak, mike, five n...",2023-10-31 21:38:53+00:00,1272341,62946,None,4766,PT2H5M16S,hd,false,Tuesday
1,RhVBRyYwrJ0,Mikecrack,¡DULCE O TRUCO! 🍬🎃 ESPECIAL de HALLOWEEN 🦇 Las...,Mike y sus amigos están listos para vivir la m...,"[mikecrack, mike crack, mikecrak, mike, hallow...",2023-10-28 18:00:07+00:00,1811885,81050,None,5392,PT11M48S,hd,false,Saturday
2,MTOsyCwbRkk,Mikecrack,¡CONSTRUYENDO UN NUEVO HOGAR! 😍🏠 Las Perrerías...,Regresan Las Perrerías de Mike Temporada 3 con...,"[mikecrack, mike crack, mikecrak, mike, las pe...",2023-10-21 20:30:08+00:00,2239161,87506,None,6674,PT8M3S,hd,false,Saturday
3,ZMt0rl2g4no,Mikecrack,🎤 TOMA TU MARTILLO 🔨😁 CANCIÓN ORIGINAL de MIKE...,Toma tu martillo es la canción original de Mik...,"[mikecrack, mike crack, mikecrak, mike, toma t...",2023-10-20 20:30:09+00:00,3229516,84065,None,8243,PT4M27S,hd,false,Friday
4,dnOv_pdLQo0,Mikecrack,EL DÍA QUE MIS SUEÑOS SE HICIERON REALIDAD... ...,Hoy os traigo un vlog muy especial en el que o...,"[mikecrack, mike crack, mikecrak, mike, la sup...",2023-10-06 02:49:18+00:00,1169296,47596,None,4947,PT17M38S,hd,false,Friday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,B81VP6ZidHo,elrubiusOMG,Skyrim - Un regalo para mi mujer,No quiere usarlo D:\r\n\r\nCancion: Dubsidia -...,"[elrubius, rubius, skyrim, mujer, comentario, ...",2012-01-04 06:30:31+00:00,1409175,41871,None,2409,PT1M35S,hd,true,Wednesday
699,ML5n5kvnbJ8,elrubiusOMG,Skyrim - La hija del Señor Oso,"No se porque tanto cabreo, si solo la acaricié!","[elrubius, rubius, skyrim, tes, señor, oso, ca...",2012-01-04 05:46:59+00:00,1393255,35706,None,892,PT41S,hd,true,Wednesday
700,b-hITqIkxD8,elrubiusOMG,Skyrim - Curiosidades divertidas -Part 1-,"Estas en pelotas, en pelotas, EN PELOTAS!\r\n\...","[elrubius, rubius, skyrim, tes, curiosidades, ...",2012-01-04 05:30:45+00:00,2262671,56350,None,2209,PT4M13S,hd,true,Wednesday
701,oD0d5xQCAYI,elrubiusOMG,Skyrim - Eligiendo a la mujer perfecta,No sirve para la cocina D:\r\n\r\nResubo!,"[elrubius, rubius, skyrim, mujer, perfecta, el...",2012-01-04 03:26:31+00:00,1874068,51256,None,2681,PT1M38S,hd,true,Wednesday
